In [10]:
import os


In [11]:
import csv
import subprocess
from tqdm import tqdm

In [12]:
def get_imageid_from_masks(mask_dir_path):
    
    ids_list = os.listdir(mask_dir_path)
    img_ids = []

    for idl in ids_list:
        t = idl.split("_")[0]
        img_ids.append(t)

    return set(img_ids)

In [13]:
def get_class_annotations(classes_list, class_descriptions_csv, annotations_csv):
    #list of selected classes
    classes = classes_list

    #Create dict with className to classId mapping
    with open(class_descriptions_csv, mode='r') as infile:
        reader = csv.reader(infile)
        dict_list = {rows[1]:rows[0] for rows in reader}
    
    csvreader = open(annotations_csv, "r")
    class_annotations = {}
    
    for ind in range(0, len(classes)):
        class_name = classes[ind]
        class_annotations[class_name] = []
        print('Class {}: {}'.format(ind, class_name))

        ##Select the annotations with backpack classID
        for line in csvreader:
            if dict_list[class_name] in line:
                class_annotations[class_name].append(line.rstrip())
                
    return class_annotations

In [5]:
def download_images_for_a_class(annotations, class_name, mask_dir_path, download_dir):
    #Create directory for downloading images

    os.makedirs(download_dir, exist_ok=True)
        
    class_annotations = annotations[class_name]
    total_annotations = len(class_annotations)
    image_id_set = get_imageid_from_masks(mask_dir_path)
    print('Number of unique IDs: {}'.format(len(image_id_set)))

    for line in class_annotations[0:total_annotations]:
        line_parts = line.split(',')
        image_id = line_parts[1]

        if image_id not in image_id_set:
            continue
            
        image_url = 's3://open-images-dataset/train/{}.jpg'.format(image_id)
        # print(image_url)
        download_path = os.path.join(download_dir, '{}.jpg'.format(image_id))
        print(download_path)
        subprocess.run(['aws', 's3', '--no-sign-request', '--only-show-errors', 'cp', image_url, download_path])
    
    return

# Data preparation 

In [6]:
# create a data root
data_root = '../../backpack_data'

os.makedirs(data_root, exist_ok=True)

In [7]:
cd_path = os.path.join(data_root, 'class-descriptions-segmentable.csv')
seg_csv_path = os.path.join(data_root, 'train-annotations-object-segmentation.csv')
backpack_image_dir = os.path.join(data_root, 'backpack-images')
backpack_mask_dir = os.path.join(data_root, 'backpack-masks')


print(f"class_df_path: {cd_path}")
print(f"seg_csv_path: {seg_csv_path}")
print(f"backpack_image_dir: {backpack_image_dir}")
print(f"backpack_mask_dir: {backpack_mask_dir}")

class_df_path: ../../backpack_data\class-descriptions-segmentable.csv
seg_csv_path: ../../backpack_data\train-annotations-object-segmentation.csv
backpack_image_dir: ../../backpack_data\backpack-images
backpack_mask_dir: ../../backpack_data\backpack-masks


In [8]:
annotations = get_class_annotations( ['Backpack'], class_descriptions_csv= cd_path, 
                                      annotations_csv= seg_csv_path)

print('Total number of annotations: {}'.format(len(annotations['Backpack'])))

Class 0: Backpack
Total number of annotations: 1082


In [9]:
# make it true to download all required data
download_unzip = True 


if download_unzip:
    download_images_for_a_class(annotations, 'Backpack', 
                                backpack_mask_dir, backpack_image_dir)

Number of unique IDs: 329
../../backpack_data\backpack-images\04feb4c5056e2f7e.jpg


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다